In [1]:
!pip install selenium beautifulsoup4
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


!pip install XlsxWriter


In [7]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

# Tạo service
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)
# Mở Google
driver.get("https://google.com")

#__________________________________________________________________________________________________
#---------- SEARCH - XỬ Lý CAPCHA (NẾU CÓ) - Truy Cập - Cuộn-----------
 
 # Chờ cho thanh tìm kiếm xuất hiện
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "gLFyf"))
)

# Tương tác với thanh tìm kiếm
input_element = driver.find_element(By.CLASS_NAME, "gLFyf")
input_element.send_keys("nguyenkim" + Keys.ENTER)

# Xử lý CAPTCHA bằng tay (nếu có capcha)
#print("Chờ bạn xử lý CAPTCHA...")
#time.sleep(30)  # Điều chỉnh thời gian tùy ý
#print("Tiếp tục sau khi CAPTCHA đã được xử lý.")

# Chờ kết quả tìm kiếm và nhấp vào liên kết
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng"))
)
click_result = driver.find_element(By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng")
click_result.click()
# Chờ trang sản phẩm tải xong
time.sleep(5)

# Cuộn đến phần tử cần lấy dữ liệu
scroll = driver.find_element(By.CLASS_NAME,'nk-product-cate-homepage')
driver.execute_script("arguments[0].scrollIntoView();", scroll)
time.sleep(1)

#-___________________________________________________________________________________________________
# ---------- Lấy link danh mục sản phẩm - link các sản phẩm trong danh mục
# Lấy link các danh mục sản phẩm
link1 = []
cl_tags = driver.find_elements(By.CLASS_NAME, 'nk-content')
for tag in cl_tags:
    try:
        a_tags = tag.find_elements(By.TAG_NAME, 'a')
        for a_tag in a_tags:
            href = a_tag.get_attribute('href')
            if href:
                link1.append(href)
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

# # Tạo một dictionary để lưu các sản phẩm theo danh mục
# products_by_category = {}

# # truy cập vào danh mục các sản phẩm
# for link in link1:
#     driver.get(link)
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid')]"))
#     )

#     # Cuộn xuống cuối trang để tải toàn bộ sản phẩm của 1 danh mục
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(3)

#     # Lấy tên danh mục sản phẩm để đặt tên sheet
#     category_name = driver.title.split('|')[0].strip()

#     # Lấy link chi tiết các sản phẩm trong danh mục
#     product_links = []
#     product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid') and contains(@class, 'nk-product-collection')]")
#     for container in product_containers:
#         try:
#             a_tags = container.find_elements(By.TAG_NAME, 'a')
#             for a_tag in a_tags:
#                 href = a_tag.get_attribute('href')
#                 if href:
#                     product_links.append(href)
#                     print(f"Link thu được: {href}")
#         except Exception as e:
#             print(f"Lỗi khi lấy link từ container sản phẩm: {e}")

#     # Lưu các link sản phẩm vào dictionary với key là tên danh mục
#     products_by_category[category_name] = product_links

# # Hàm loại bỏ ký tự không hợp lệ trong tên sheet 
# # ( vì trong sheet không cho đặt tên bằng các ký tự đặc biệt)
# def clean_sheet_name(sheet_name):
#     return re.sub(r'[\\/:*?"<>|]', '', sheet_name)[:31] # sheet có tối đa 31 ký tự
# #________________________________________________________________________________________________________
# # ------------------------- Lấy thông tin từng sản phẩm --------------------------------------
# sanpham = []
# for linksp in link1:
#     try:
#         driver.get(linksp)
#         time.sleep(4)
#          # Cuộn xuống cuối trang để chắc chắn toàn bộ thông tin sản phẩm đã tải
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(2)

#         # Trích xuất tên sản phẩm
#         try:
#             namesp = driver.find_element(By.CLASS_NAME,"product-title").text
#         except:
#             namesp=''

#         # Trích xuất giá bán 
#         try:
#             price = driver.find_element(By.CLASS_NAME,"final-price").text
#         except:
#             price=''
        
#         # Trích xuất các thông tin khác
#         try:
#              info_elements = driver.find_elements(By.CLASS_NAME, "group")  # Dùng find_elements để lấy nhiều phần tử
#              info = '\n'.join([element.text for element in info_elements])  # Kết hợp tất cả thông tin thành chuỗi
#         except:
#             info = ''
#         # Thêm thông tin vào danh sách 
#         sanpham.append({'NAME': namesp, 'PRICE':price , 'INFORMATION': info})
        
#     except Exception as e:
#         print(f"Error processing {linksp}: {e}")  
       
        
#        # __________________________________________________________________________
# # ---------------------- TẠO DATAFRAME  -------------------------------------------
# # Tạo DataFrame và lưu từng danh mục vào một sheet riêng
# file_name = "products_by_category.xlsx"
# with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
#     for category, links in products_by_category.items():
#         if isinstance(links, list) and links:  # Kiểm tra nếu links là danh sách và không rỗng
#             valid_links = [link for link in links if isinstance(link, str)]
#             if valid_links:
#                 cleaned_name = clean_sheet_name(category)  # Làm sạch tên sheet
#                 if cleaned_name:  # Kiểm tra nếu cleaned_name không rỗng
#                     df = pd.DataFrame(valid_links, columns=['Link Sản Phẩm'])
#                     df.to_excel(writer, sheet_name=cleaned_name, index=False)
#                 else:
#                     print(f"Tên danh mục '{category}' sau khi làm sạch không hợp lệ.")
#             else:
#                 print(f"Danh mục '{category}' không có liên kết hợp lệ.")
#         else:
#             print(f"Danh mục '{category}' không có dữ liệu hoặc không phải là danh sách.")
# print(f"Đã lưu thành công vào {file_name}")


# # Tạo data frame sanpham.xlsx
# sanpham_df =pd.DataFrame(sanpham)
# # Lưu vào file EXcel
# file_name="SP.xlsx"
# sanpham_df.to_excel(file_name,index=False)
# print('ĐÃ lưu thành công các sản phẩm ')

# #___________________________________________________________________________________________________
# # Đóng trình duyệt
# driver.quit()


In [8]:
link1

['https://www.nguyenkim.com/may-lanh/',
 'https://www.nguyenkim.com/dien-thoai-di-dong/',
 'https://www.nguyenkim.com/tu-lanh/',
 'https://www.nguyenkim.com/tivi/',
 'https://www.nguyenkim.com/may-tinh-xach-tay/',
 'https://www.nguyenkim.com/may-giat/',
 'https://www.nguyenkim.com/quat-vi/?features_hash=65-148987-148986-5954',
 'https://www.nguyenkim.com/may-nuoc-nong/',
 'https://www.nguyenkim.com/may-in-van-phong/',
 'https://www.nguyenkim.com/noi-com-dien/',
 'https://www.nguyenkim.com/noi-chien-khong-dau/',
 'https://www.nguyenkim.com/may-loc-khong-khi/',
 'https://www.nguyenkim.com/man-hinh-lcd-vi-tinh/',
 'https://www.nguyenkim.com/may-ep-trai-cay/',
 'https://www.nguyenkim.com/may-loc-nuoc/',
 'https://www.nguyenkim.com/quat-vi/',
 'https://www.nguyenkim.com/bep-tu-hong-ngoai',
 'https://www.nguyenkim.com/amply-va-loa/',
 'https://www.nguyenkim.com/may-tinh-bang/',
 'https://www.nguyenkim.com/may-say-toc/']

In [9]:

product_links = []
# Tạo vòng lập để duyệt qua từng link
for link in link1:
    driver.get(link)
    while True:
        try:    
            action_1 = ActionChains(driver)
            # Cuộn xuống trang để tải toàn bộ sản phẩm
            scroll_frame = driver.find_element(By.ID, "feature-product-grid")
            action_1.move_to_element(scroll_frame).perform()

            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid')]"))
            )
            # Lấy các container sản phẩm
            product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid') and contains(@class, 'nk-product-collection')]")
                        
            for container in product_containers:
                try:
                    # Tìm tất cả các thẻ <a> trong container
                    a_tags = container.find_elements(By.TAG_NAME, 'a')
                    for a_tag in a_tags:
                        href = a_tag.get_attribute('href')
                        if href and href not in product_links:  # Kiểm tra để không thêm trùng link
                            product_links.append(href)
                            print(f"Link thu được: {href}")        
                except Exception as e:
                    print(f"Lỗi khi lấy link từ container sản phẩm: {e}")
                # Tìm nút Next Page và click để chuyển sang trang tiếp theo
                try:
                    action_2 = ActionChains(driver)
                    move_frame_next_page = driver.find_element(By.CLASS_NAME,'ty-pagination NkReview_footer clearfix')
                    action_2.move_to_element(move_frame_next_page).perform()
                    next_page = driver.find_element(By.CLASS_NAME, 'NkReview_footer_col-3')
                    next_page.click()                    
                    print("Chuyển sang trang tiếp theo...")
                    time.sleep(3)  # Chờ một vài giây để trang tiếp theo tải xong
                except Exception as e:
                    print(f"Lỗi khi click nút Next: {e}")
        except:
            print("Lỗi thu thập")
            break        
            
            



Link thu được: https://www.nguyenkim.com/may-lanh-casper-inverter-tc-09is35.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=128074
Link thu được: https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-pu12zkh-8m.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=127689
Link thu được: https://www.nguyenkim.com/may-lanh-reetech-1-hp-rt9-rc9-ta-bt.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=128121
Link thu được: https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-xu9zkh-8.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=127668
Link thu được: https://www.nguyenkim.com/may-lanh-daikin-ftky25wmvmv.html
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=116296
Link thu được: https://www.nguyenkim.com/may-lan

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.101)
Stacktrace:
	GetHandleVerifier [0x00007FF640F1FDA5+29557]
	(No symbol) [0x00007FF640E92240]
	(No symbol) [0x00007FF640D4B6EA]
	(No symbol) [0x00007FF640D1FCD5]
	(No symbol) [0x00007FF640DCF167]
	(No symbol) [0x00007FF640DE81A1]
	(No symbol) [0x00007FF640DC72A3]
	(No symbol) [0x00007FF640D912DF]
	(No symbol) [0x00007FF640D92451]
	GetHandleVerifier [0x00007FF64124DCBD+3363469]
	GetHandleVerifier [0x00007FF641299B47+3674391]
	GetHandleVerifier [0x00007FF64128EAEB+3629243]
	GetHandleVerifier [0x00007FF640FDFC66+815670]
	(No symbol) [0x00007FF640E9D6EF]
	(No symbol) [0x00007FF640E992B4]
	(No symbol) [0x00007FF640E99450]
	(No symbol) [0x00007FF640E881FF]
	BaseThreadInitThunk [0x00007FFB1B657374+20]
	RtlUserThreadStart [0x00007FFB1C0BCC91+33]


In [20]:
len(product_links)

1174